In [1]:
import sys
import inspect
from jw_utils import jw_draw_tree as jdt
from ete3 import ncbi_taxonomy
from ete3 import Tree
ncbi_tax = ncbi_taxonomy.NCBITaxa()
import json
from jw_utils import ncbi_datasets_fxs as nfx
from orthofinder_utils import dash_ortho_parser as dop
from Bio import Phylo
from plotly import graph_objects as go
from jw_utils import jw_ncbi_taxonomy as jnt
import os
from jw_utils import plotly_preferences as pprefs
pprefs.__file__
import pandas as pd
from orthofinder_utils import itol_annotations as itol_annots

In [2]:
rgb_to_hex = {'rgba(132,137,145,1)':'#848991',
 'rgb(0,208,132)':'#00d084',
 'rgb(0,122,255)':'#007aff',
 'rgb(171,184,195)':'#abb8c3',
 'rgb(255,105,0)':'#ff6900',
 'rgb(252,185,0)':'#fcb900',
 'rgb(123,220,181)':'#7bdcb5',
 'rgb(142,209,252)':'#8ed1fc',
 'rgb(6,147,227)':'#0693e3',
 'rgb(155,81,224)':'#9b51e0',
 'rgb(207,46,46)':'#cf2e2e',
 'rgb(247,141,167)':'#f78da7'}

hex_colors = list(rgb_to_hex.values())

In [35]:
df = pd.read_csv('./all_strains_summary.tsv', sep='\t')[['Strain ID', 'Full organism name', 'Species',
       'Strain', 'Assembly level', 'Host taxonomy ID', 'Host Disease', 
       'Isolation Source', 'Assembly name', 'Refseq assembly accession',
       'Scaffold N50', 'Contig N50']]
df_disease = df.loc[df['Host Disease']=='cystic fibrosis',:]
df_disease = df_disease.loc[df['Assembly level']=='Complete Genome',:]
CF_accessions = list(df_disease['Refseq assembly accession'])
df_disease = df_disease.set_index('Refseq assembly accession')

In [6]:
HOGs = {
    'crc':'N3.HOG0004558', 'cbrA':'N3.HOG0005021', 'pelA':'N3.HOG0009322','pelB':'N3.HOG0009282','pelC':'N3.HOG0009225','cupB1':'N3.HOG0006832',
    'cupB2':'N3.HOG0001264','cupB3':'N3.HOG0001773','cupC1':'N3.HOG0007291','cupC2':'N3.HOG0001265','cupC3':'N3.HOG0001776',
    }
HOGs_truncated = {
    'cbrA':'N3.HOG0005021', 'crc':'N3.HOG0004558', 'pelA':'N3.HOG0009322','cupB1':'N3.HOG0006832','cupC2':'N3.HOG0001265',
     }



In [7]:
CF_accessions
with open('./CF_accessions.txt', 'w') as f:
    for acc in CF_accessions:
        f.write(acc + '\n')
#!datasets download genome accession --inputfile ./CF_accessions.txt 

In [8]:
dop_obj = dop.DashOrthoParser('./data', tax_level='N3')
tree =  Phylo.read('./data/Species_Tree/SpeciesTree_rooted_node_labels.txt', format='newick')
HOGs_to_name_d = {k:v for v,k in HOGs_truncated.items()}
names, binary_count_d = itol_annots.make_binary_count_dict(tree, HOGs_truncated.values(),dop_obj)

In [11]:
itol_outfile_path = './itol_tree/multibar_dataset_trunc.txt'
itol_annots.make_multi_itol_bargraph_dataset('./itol_tree/multibar_dataset_trunc.txt',binary_count_d, names)

GCF_002362295.1,0,0,0,0,0
GCF_000046845.1,0,0,0,0,0
GCF_005281455.1,0,0,0,0,0
GCA_001077675.1,0,0,0,0,0
GCF_000413935.1,0,0,0,0,0
GCF_001682515.1,0,0,0,0,0
GCF_900105655.1,1,1,0,0,0
GCF_900105255.1,1,1,1,0,0
GCF_900105005.1,1,1,0,0,0
GCF_009497895.1,1,1,0,0,0
GCF_020025155.1,1,1,1,0,0
GCF_900104945.1,1,1,1,0,0
GCF_008693965.1,1,1,0,0,0
GCF_014522265.1,1,1,1,0,0
GCF_001518815.1,1,1,1,0,0
GCF_900105995.1,1,1,0,0,0
GCF_900105885.1,1,1,0,0,0
GCF_900104805.1,1,1,0,0,0
GCF_024448735.1,1,1,1,0,0
GCF_021432765.1,1,1,0,0,0
GCF_009740185.1,1,1,0,0,0
GCF_000327065.1,1,1,0,0,0
GCF_000825645.2,1,1,0,0,0
GCF_900636845.1,1,1,0,0,0
GCF_000013785.1,1,1,0,0,0
GCF_000219605.1,1,1,0,0,0
GCF_021605025.1,1,1,0,1,0
GCF_019879085.1,1,1,0,1,0
GCF_011397855.1,1,1,0,1,0
GCF_020422905.1,1,1,0,1,0
GCF_013393345.1,1,1,0,1,1
GCF_001708505.1,1,1,0,0,1
GCF_009911755.1,1,1,0,1,0
GCF_004745455.1,1,1,0,1,0
GCF_000349845.1,1,1,0,1,0
GCF_900188285.1,1,1,0,0,0
GCF_012971725.1,1,1,0,0,0
GCF_000017205.1,1,1,1,1,1
GCF_02255954

In [42]:
accessions_trunc = []
for acc in CF_accessions:
    if dop_obj.accession_to_name.get(acc):
        print(dop_obj.accession_to_name.get(acc))
        accessions_trunc.append(acc)
df_disease_trunc = df_disease.loc[accessions_trunc, :][['Full organism name', 'Species', 'Strain', 'Assembly level', 'Host Disease', 'Isolation Source']]

Pseudomonas aeruginosa DK2
Pseudomonas aeruginosa SCV20265
Pseudomonas fulva FDAARGOS_167
Pseudomonas monteilii FDAARGOS_171
Pseudomonas aeruginosa AES1M
Pseudomonas aeruginosa AES1R
Pseudomonas aeruginosa DVT419
Pseudomonas aeruginosa DVT779
Pseudomonas aeruginosa DVT412
Pseudomonas aeruginosa PES_P749


In [44]:
df_disease_trunc.to_csv('./Clinical_isolates.csv')

In [19]:
relabel_dict = {}
for acc, name in dop_obj.accession_to_name.items():
    name = name.replace('[','').replace(']','')
    name_list =  name.split(' ')
    genus = name_list[0][:1]+'.'
    species = name_list[1]
    #strain = ' '.join(name_list[2:])[:10]
    strain = name_list[-1][:10]
    name = f'{genus} {species} {strain}' 
    relabel_dict[acc]=name

In [21]:

outfile_path = './itol_tree/rename_dataset.txt'
itol_annots.relabel_itol_treeleafs(tree, relabel_dict,outfile_path)

In [16]:
    
outfile_path = './itol_tree/rename_dataset.txt'
#relabel_itol_treeleafs(tree, relabel_dict, outfile_path)

In [192]:
import shutil
dest = './Proteomes'
parent_fp = './data/ncbi_dataset/data/'
for folder in [f for f in os.listdir(parent_fp) if f.startswith('GC')]:
    proteome_fp = os.path.join(parent_fp,folder, 'sequence_report.jsonl')
    if os.path.exists(proteome_fp):
        n_dest_fp = os.path.join(dest, folder)
        shutil.move(proteome_fp, n_dest_fp)

In [49]:
def make_simple_itol_bargraph_dataset(template_path, file_path, data_dict):
    """
    Add data to simple bargraph template for itol dataset adn write to file.

    Parameters
    ----------
    template_path : str
        path to template file from itol.
    file_path : str
        path for new file that is to be created
    data_dict :dict
        {terminal node name: count}

    """


    template_lines = []
    with open(template_path, 'r') as f:
        for line in f:
            if line[-1] != '\n':
                line = line+'\n'
            template_lines.append(line)
            
    with open(file_path, 'w') as f:
        for line in template_lines:
            f.write(line)
        for name, count in data_dict.items():
            f.write(f'{name},{count}\n')
            
crc_data = list(copynumber_bargraph_data_dict([HOGs['crc']], dop_obj, id_type='accession').values())[0]
template_path = './itol_tree/dataset_multibar_template.txt'
file_path = './itol_tree/crc_simplebar_dataset.txt'
make_simple_itol_bargraph_dataset(template_path, file_path, crc_data)